# 📊 Financial Performance Snapshot  
## Data Exploration Notebook

### Project Context
This project simulates a real-world analytics engagement for a small-to-medium business.  
The objective is to assess **financial performance over a recent three-month period** using transactional sales data.

The analysis focuses on understanding:
- Revenue generation
- Cost structure
- Margin dynamics
- Key operational drivers by category and payment method

This notebook represents **Phase 1: Data Exploration**, whose purpose is to understand the structure, quality, and limitations of the raw data **before any transformation or KPI computation**.

---

### Business Objective
Provide a clear and reliable **financial snapshot** that answers the following executive-level questions:

- How much revenue is the business generating?
- What are the main cost and margin drivers?
- How does performance vary by category and payment method?
- What data limitations must be considered before decision-making?

---

### Scope of This Notebook
In this phase, we will:

- Load and inspect the raw transactional dataset
- Understand the time coverage and granularity of the data
- Identify missing, inconsistent, or potentially invalid records
- Document data quality issues and assumptions

**No data cleaning or KPI calculations are performed in this notebook.**

---

### Dataset Description
The dataset contains transactional sales records with the following key fields:

- Order identifier
- Transaction date
- Product or service category
- Payment method
- Revenue amount
- Estimated cost per transaction

The data represents daily transactions over a three-month period and includes intentional imperfections to reflect real-world data conditions.

---

### Expected Outputs
By the end of this notebook, we will have:

- A clear understanding of the dataset structure
- A documented list of data quality issues
- Initial hypotheses to guide the cleaning and KPI analysis phases

These findings will inform the **data cleaning rules** and **financial KPI definitions** used in subsequent steps.

---

## 1) Load & Basic Inspection
**Objective**

Confirm that the dataset loads correctly, understand its overall structure and size, and verify that the data represents transaction-level records suitable for financial analysis.

In [24]:
import pandas as pd
import numpy as np

# load data and check shape
financial_data = pd.read_csv("../data/raw/financial_transactions_raw.csv")
financial_data.shape

(500, 6)

In [25]:
# Inspect first 5 rows
financial_data.head()

,order_id,order_date,category,payment_method,revenue,cost
0,1001,2024-02-21,Electronics,Credit Card,728.90,530.75
1,1002,2024-01-15,Electronics,Credit Card,587.32,422.64
2,1003,2024-03-12,Electronics,Cash,136.99,89.15
3,1004,2024-03-01,Home,Credit Card,223.49,156.03
4,1005,2024-01-21,Electronics,Cash,709.37,483.00


In [26]:
# Inspect last 5 rows
financial_data.tail()

,order_id,order_date,category,payment_method,revenue,cost
495,1496,2024-03-07,Home,Debit Card,843.71,523.70
496,1497,2024-03-05,NaN,Cash,617.61,427.80
497,1498,2024-02-02,Home,Credit Card,121.08,76.35
498,1499,2024-02-09,Clothing,Cash,921.79,553.04
499,1500,2024-03-14,Electronics,Credit Card,85.12,60.28


In [27]:
# Inspect 5 random rows
financial_data.sample(5, random_state=42)

,order_id,order_date,category,payment_method,revenue,cost
361,1362,2024-03-30,Clothing,Credit Card,458.53,203.05
73,1074,2024-03-03,Electronics,Credit Card,95.93,73.53
374,1375,2024-02-29,Home,Credit Card,463.45,247.18
155,1156,2024-01-29,Clothing,Credit Card,70.66,36.30
104,1105,2024-03-28,Home,Credit Card,301.95,159.65


### Load & Basic Inspection — Initial Observations

- The dataset contains **500 transactional records** with multiple financial and categorical fields.
- Each row represents a single customer transaction.
- The data appears to be at a **transaction-level granularity**, not aggregated.
- Initial inspection shows a mix of numerical and categorical fields relevant for financial analysis.

Further inspection is required to assess data types, missing values, and consistency.

## 2) Data types & Schema Validation
**Objective**

Validate that each field has an appropriate data type for financial analysis and identify schema issues that may impact downstream KPI computation.

In [28]:
# Inspect data types
financial_data.dtypes

order_id            int64
order_date         object
category           object
payment_method     object
revenue           float64
cost              float64
dtype: object

In [32]:
# Check missing values (count and percentage)
missing_summary = pd.DataFrame({
    'missing_count': financial_data.isna().sum(),
    'missing_pct': financial_data.isna().mean() * 100
}).sort_values(by='missing_count', ascending=False)

missing_summary

,missing_count,missing_pct
revenue,10,2.0
category,8,1.6
order_date,5,1.0
order_id,0,0.0
payment_method,0,0.0
cost,0,0.0


In [34]:
# Validate categorical field consistency
financial_data['category'].value_counts(dropna=False)

category
Electronics    159
Home           148
Clothing       120
Services        65
NaN              8
Name: count, dtype: int64

In [35]:
financial_data['payment_method'].value_counts(dropna=False)

payment_method
Credit Card    201
Debit Card     133
Cash           112
Transfer        54
Name: count, dtype: int64

### Data Types & Schema Validation — Observations

- `order_date` is currently loaded as an object and can be safely converted to a datetime format, with a limited number of invalid or missing entries.
- Financial fields (`revenue`, `cost`) are numeric; however, **revenue contains missing values while cost does not**, indicating the presence of transactions with recorded costs but no associated revenue.
- This inconsistency represents a **material data quality issue**, as revenue is required for all profitability and margin-related KPIs.
- Categorical fields (`category`, `payment_method`) show a small, well-defined set of values but include missing entries that will require standardization.

No data transformations have been applied yet. 

## 3) Time Coverage & Granularity
**Objective**

Validate the time range covered by the dataset, confirm transaction granularity, and identify gaps or irregular activity that may affect trend analysis.

In [37]:
# Inspect date range (using safe conversion)
order_dates = pd.to_datetime(financial_data['order_date'], errors='coerce')

order_dates.min(), order_dates.max()

(Timestamp('2024-01-01 00:00:00'), Timestamp('2024-03-31 00:00:00'))

In [41]:
# Count transactions by month
transactions_by_month = (order_dates.dt.to_period('M').value_counts())

transactions_by_month

order_date
2024-01    171
2024-02    169
2024-03    155
Freq: M, Name: count, dtype: int64

In [43]:
# Check daily transaction density
transactions_by_day = order_dates.value_counts()

transactions_by_day.describe()

count    91.000000
mean      5.439560
std       2.700653
min       1.000000
25%       3.000000
50%       5.000000
75%       7.000000
max      14.000000
Name: count, dtype: float64

In [46]:
# Identify days with no transactions
full_date_range = pd.date_range(
    start=order_dates.min(),
    end=order_dates.max(),
    freq='D'
)

missing_days = full_date_range.difference(order_dates.dropna().unique())

len(missing_days)

0

In [50]:
# Check transaction density variation
transactions_by_day.sort_values()

order_date
2024-01-31     1
2024-03-23     1
2024-03-19     2
2024-03-14     2
2024-01-10     2
              ..
2024-01-02    11
2024-01-24    11
2024-03-30    12
2024-03-02    14
2024-02-02    14
Name: count, Length: 91, dtype: int64

### Time Coverage & Granularity — Observations

- The dataset covers a continuous period of approximately **three months**, from early January to late March 2024.
- Transactions are recorded at a **transaction-level granularity**, with multiple records per day.
- All calendar days within the observed date range contain at least one recorded transaction, indicating continuous operational activity.
- Transaction volumes vary across days, reflecting uneven daily activity levels rather than data gaps.
- A limited number of records contain missing transaction dates and will require handling during the data cleaning phase.

Overall, the time coverage and granularity are sufficient to support short-term performance analysis and month-over-month KPI evaluation.

## 4) Missing Values Analysis
**Objective**

Quantify missing data across the dataset, assess its materiality, and identify fields that require explicit handling before financial KPI computation.

In [53]:
# Missing values summary (count and percentage) - from 2) Data Types & Schema Validation
missing_summary

,missing_count,missing_pct
revenue,10,2.0
category,8,1.6
order_date,5,1.0
order_id,0,0.0
payment_method,0,0.0
cost,0,0.0


In [55]:
# Inspect records with missing revenue
financial_data.loc[
    financial_data['revenue'].isna(),
    ['order_id', 'order_date', 'category', 'payment_method', 'cost']
].head()

,order_id,order_date,category,payment_method,cost
66,1067,2024-02-19,Clothing,Transfer,170.57
88,1089,2024-03-25,Electronics,Cash,262.17
211,1212,2024-01-01,Clothing,Transfer,595.63
221,1222,2024-02-24,Home,Credit Card,53.33
235,1236,NaN,Electronics,Debit Card,458.58


In [57]:
# Check whether missing revenue is concentrated in specific categories or payment methods
financial_data.loc[financial_data['revenue'].isna(),['category', 'payment_method']].value_counts()

category     payment_method
Clothing     Transfer          2
Home         Credit Card       2
Electronics  Debit Card        2
             Cash              1
Clothing     Credit Card       1
Electronics  Credit Card       1
Home         Debit Card        1
Name: count, dtype: int64

### Missing Values — Observations

- Missing values are present across a limited number of fields, with `revenue` and `order_date` being the most relevant for financial analysis.
- Approximately **2% of transactions** have missing revenue values, while all records contain cost information.
- Missing revenue does not appear to be heavily concentrated in a single category or payment method.
- A small number of records contain missing transaction dates, which may affect time-based aggregations.

No imputation or record removal has been performed at this stage. 

## 5) Duplicates & Uniqueness
**Objective**

Verify transactional integrity by ensuring that each record represents a unique transaction and that no duplicate records inflate financial metrics.

In [59]:
financial_data['order_id'].is_unique

True

In [60]:
# Count duplicated order IDs
financial_data['order_id'].duplicated().sum()

np.int64(0)

In [62]:
# Check for fully duplicated rows
financial_data.duplicated().sum()

np.int64(0)

### Duplicates & Uniqueness — Observations

- Each transaction is identified by a unique `order_id`.
- No duplicated order identifiers were detected in the dataset.
- No fully duplicated rows were found.
- Transactional integrity appears to be preserved, and no revenue inflation risk due to duplication was identified.

No deduplication actions are required prior to KPI computation.

## 6) Basic Numerical Sanity Checks
**Objective**

Detect impossible or suspicious numerical values that could distort financial KPIs and profitability analysis.

In [63]:
# Check for zero or negative revenue values
financial_data.loc[
    financial_data['revenue'] <= 0,
    ['order_id', 'order_date', 'category', 'revenue', 'cost']
].shape

(0, 5)

In [64]:
# Check for zero or negative cost values
financial_data.loc[
    financial_data['cost'] <= 0,
    ['order_id', 'order_date', 'category', 'revenue', 'cost']
].shape

(0, 5)

In [65]:
# Identify transactions where cost exceeds revenue
financial_data.loc[
    financial_data['cost'] > financial_data['revenue'],
    ['order_id', 'order_date', 'category', 'revenue', 'cost']
].shape

(0, 5)

In [67]:
# High-level distribution check for revenue and cost
financial_data[['revenue', 'cost']].describe()

,revenue,cost
count,490.000000,500.000000
mean,295.288469,184.375280
std,207.314191,137.842179
min,10.720000,6.660000
25%,138.790000,84.895000
50%,254.245000,148.880000
75%,409.792500,247.967500
max,1185.020000,915.990000


### Basic Numerical Sanity Checks — Observations

- No transactions with zero or negative revenue were identified.
- No transactions with zero or negative cost values were detected.
- No transactions with costs exceeding revenue were identified.
- Revenue and cost distributions appear reasonable for transaction-level financial data, with no extreme values that would prevent analysis.

Overall, numerical values are largely plausible.

## 7) Observed Data Issues & Assumptions

### Key Data Quality Issues
- A small percentage of transactions contain **missing revenue values**, while associated costs are present. These records cannot be used directly for profitability or margin calculations.
- A limited number of transactions have **missing transaction dates**, which may affect time-based aggregations if not handled explicitly.
- Categorical fields contain **missing values**, though no structural inconsistencies or unexpected categories were observed.
- No duplicated transactions or order identifiers were detected.

### Analytical Assumptions
- Each row represents a **single, independent transaction**.
- Recorded revenue and cost values reflect gross transaction amounts prior to any aggregation.
- Missing or invalid records will be handled using **explicit, documented cleaning rules** rather than implicit imputation.
- Financial KPIs will be computed only on records meeting defined validity criteria.

### Risks & Limitations
- Excluding transactions with missing revenue may slightly understate total revenue and margins.
- Cost estimates may introduce noise into margin calculations.
- The analysis supports **short-term performance evaluation** and should not be interpreted as evidence of long-term trends or seasonality.

These observations inform the data cleaning rules and KPI definitions applied in subsequent analysis stages.